In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import math

from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib as mpl
from matplotlib import pyplot as plt
from pylab import rcParams
import networkx as nx

from setup import Graph
from sklearn.metrics import accuracy_score
import operator


In [32]:
class Graph: 
        
    def __init__(self, file_name):
        self.file_name = file_name
        
    
    def make_nodes(self,target_list):
        result = []
        for i in range(len(target_list)-1):
            a = (target_list[i], target_list[i+1])
            result.append(a)
        return result

    
    def create_listEdges(self):
        
        data = pd.read_csv(f'data/{self.file_name}.csv')
        
        if self.file_name in ['bpic13', 'hospitalbilling', 'helpdesk']:
            if self.file_name == 'bpic13':
                activities= {k:v+1 for v, k in enumerate(sorted(set(data.loc[:,'Activity'])))}
                data.loc[:,'ActivityID'] = [activities[i] for i in data.Activity]
                data.rename(columns = {'Case ID' : 'CaseID'}, inplace = True)

            elif self.file_name == 'hospitalbilling':
                activities= {k:v+1 for v, k in enumerate(sorted(set(data.loc[:,'activity'])))}
                data.loc[:,'ActivityID'] = [activities[i] for i in data.activity]
                data.rename(columns = {'case_id' : 'CaseID'}, inplace = True)

            elif self.file_name == 'helpdesk':

                activities= {k:v+1 for v, k in enumerate(sorted(set(data.loc[:,'ActivityID'])))}
            
            
            no_Next = data.groupby('CaseID').filter(lambda x: x['ActivityID'].count() == 1.)['CaseID'].tolist()

            for i in no_Next:
                data.drop(data[data['CaseID'] == i].index, inplace = True )
                
            act_list = activities.values()
            actname_list = activities.keys()

            Dict_act = {}

            for x, y in zip(act_list, actname_list) :
                Dict_act[x] = y

            list_activities = data.groupby('CaseID')['ActivityID'].apply(list)
            list_activities.reset_index( inplace = True, drop=True )

            target = list_activities.apply(lambda x : list(map(str, x)))
            list_tuples = target.apply(lambda x: self.make_nodes(x) )
            
            
            ##### Description of Variables
            # list_activities : [1,2,3,...] => list of activities in each case
            # list_tuples: [(1,2),(2,3)---] => Edge list
            # new_activities : {'1' : act_name, '2' : act_name2}  => Dict of Activity ID : Activity name 
            # act_list : [1, 2 ...]  => List of act_id

            self.list_activities = list_activities
            self.list_tuples = list_tuples               
            self.Dict_act = Dict_act 
            self.act_list = act_list

            return self.list_activities
            
            
        else:
            
            cols = []

            file_ = self.file_name.split('_')[0]
            prefix = int(self.file_name.split('_')[1])


            ##### Update the name of columns depending on Prefix #####
            data.drop(data.columns[-2], axis=1, inplace = True)
            cols = [f'Activity{i+1}' for i in range(prefix)] 
            cols.extend(['Next_Activity'])

            ID_cols = [f'Activity{i+1}_ID' for i in range(prefix)] 
            ID_cols.extend(['Next_Activity_ID'])


            data.columns = cols

            if file_ == 'hospitalbilling' and prefix == 3: 
                act_list = set()
                for i in range(prefix): 
                    target = f'Activity{i+1}'
                    tmp = set(data[target])
                    act_list = act_list | tmp 
                    
            else: 
                ##### Take the unique activity list and Dict of act & act_id #####
                act_list = set(data['Next_Activity'])
                for i in range(prefix): 
                    target = f'Activity{i+1}'
                    tmp = set(data[target])
                    act_list = act_list | tmp 
                    
            activities = {k:v+1 for v, k in enumerate(sorted(act_list))}

            act_list = activities.values()
            actname_list = activities.keys()

            Dict_act = {}

            for x, y in zip(act_list, actname_list) :
                Dict_act[x] = y

            if  file_ == 'hospitalbilling' and prefix == 3: 
                cols.remove('Next_Activity')
                ID_cols.remove('Next_Activity_ID')

            for idx_col, col in enumerate(cols) :
                data.loc[:,ID_cols[idx_col]] = [activities[i] for i in data.loc[:,col]]
                
                
            ##### Take activity list by Case #####
            if  file_ == 'hospitalbilling' and prefix == 3: 
#                 cols.remove('Next_Activity')
                ID_cols.append('Next_Activity')
                list_activities = data[ID_cols].values.tolist()
            
            else:
                list_activities = data.values.tolist()
            

            list_activities = data[ID_cols].values.tolist()
            list_activities = pd.Series( list_activities )

            ##### Make Tuples to create edges in the Graph #####
            target = list_activities.apply(lambda x : list(map(str, x)))
            list_tuples = target.apply(lambda x: self.make_nodes(x) )
            

            self.list_activities = list_activities
            self.list_tuples = list_tuples               
            self.Dict_act = Dict_act 
            self.act_list = act_list


            return self.list_activities

        
    
    def make_graph(self, target_activity):

        G = nx.DiGraph()

        for tuples in self.list_tuples:
            if len(tuples) == 0: 
                continue
            else: 
                for i in range(len(tuples)):

                    (find_u, find_v)  = tuples[i]

                    if find_u == target_activity:
                        G.add_node(target_activity, level=0)

                        # target activity으로 시작하는 tuple을 찾아서 새로운 list 생성
                        new_tuples = tuples[i:]
                        nodes = [target_activity]

                        if len(new_tuples)==0:
                            continue
                        else:
                            # new_tuples의 edges를 그려줌
                            for idx, tpl in enumerate(new_tuples):

                                (real_s,real_e) = tpl
                                current_nodes = list(G.nodes())
                                current_edges= list(G.edges)

                                if idx ==0:
                                    s = real_s

                                e = f'L{idx+1}_{real_s}-{real_e}'   

                                if (s,e) not in current_edges:
                                    G.add_node(e, level=idx+1)
                                    G.add_edges_from([(s,e)], weight =1)

                                else:
                                    G.add_edges_from([(s,e)], weight = G.edges[s,e]['weight'] +1, level = idx)

                                s = e

                    else:
                        continue

        return G
    
    
    def convertWeignttoProbability(self):
    
        g_cnt = len(self.list_G)

        for G in self.list_G:

            nodes = list(G.nodes)

            for node in nodes:
                outs = list(G.out_edges([node]))
                weight_list = [G[u][v]['weight'] for u,v in outs]
                total = np.sum(weight_list)

                for u,v in outs:
                    result = G[u][v]['weight']/total
                    G[u][v]['weight'] = round(result,5)

        
    def graphs(self):
    
        self.create_listEdges()
    #     print(list_tuples)

        list_G = []

        for i in self.act_list:

            ##### Create G for the target activity #####
            target_act = f'{i}'
            G = self.make_graph(target_act)

            list_G.append(G)
        
        self.list_G = list_G
        self.convertWeignttoProbability()
        
        return self.list_G , self.list_activities
    
    def take_pos(self, G):
        pos_dict = {}
        levels = {info['level'] for n,info in G.nodes(data=True)}

        for i, ll in enumerate(range(min(levels), max(levels)+1)):
            nodes = [n for n,info in G.nodes(data=True) if info['level']==ll]
            y = 1.0 - (1.0/(len(levels)+1)*(i+1))
            pos_dict.update({n:(-(y*100),1.0/(len(nodes)+1)*(j+1)*200) for j, n in enumerate(nodes)})
        return pos_dict
    
    def Draw_Graphs(self):
    
        for G in self.list_G: 
                       
            target_event = list(G.nodes)[0]

            ##### Edge Labels #####
            edge_labels=dict([((u,v,),d['weight']) for u,v,d in G.edges(data=True)])

            ##### Node Color by level #####
            # Setting for Color
            color_lookup = {k:G.nodes(data=True)[k]['level'] for v, k in enumerate(sorted(set(G.nodes())))}
            low, *a , high = sorted(color_lookup.values())
            norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
            mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.coolwarm)

            # Setting color for each level
            level_colors = {i : mapper.to_rgba(i) for i in set(color_lookup.values())}
            colors = [level_colors[G.nodes(data=True)[node]['level']] for node in G.nodes()]

            ##### Draw G #####
            plt.figure(figsize=(24, 24))
            nx.draw_networkx(G, pos=self.take_pos(G), node_color=colors, node_size=2000, with_labels = True, font_size = 16)   
            nx.draw_networkx_edge_labels(G, pos=self.take_pos(G),edge_size= 100,edge_labels=edge_labels, font_size= 16)
            plt.title(f'{self.file_name}_{target_event}')
            plt.savefig(f'Result_Graphs/{self.file_name}-Event({target_event}) Graph.png')
            plt.close()


In [24]:
class prediction:
    
    def __init__(self, list_G,list_activities):
        self.list_G = list_G
        self.list_activities = list_activities
        
        
    def make_nodes(self,target_list):
        
        result = []
        for i in range(len(target_list)-1):
            a = (target_list[i], target_list[i+1])
            result.append(a)
        return result

    
    def activity_seperation(self):

        prev_act = self.list_activities.apply(lambda x: x[:-1])
        self.prev_act = prev_act.apply(lambda x : list(map(str, x)))
        last_act = self.list_activities.apply(lambda x: x[-1:])
        self.last_act = last_act.apply(lambda x : list(map(str, x)))
        
        return self.list_activities, self.prev_act,self.last_act 
        
        
    def make_edgeTuples(self, target): 
        
        list_tuples = target.apply(lambda x: self.make_nodes(x) )
        return list_tuples
    
    
    # function to find the candidate nodes and the weight prob
    def result_nodewithProb(self,initial_S,e):

        weight = 1 
        candicate_weights = []

        candicate_nodes = list(self.list_G[initial_S][e])

        for find_e in candicate_nodes:
            c_weight = self.list_G[initial_S].edges[e,find_e]['weight']
            candicate_weights.append(c_weight)

        candicate_nodes = [i[-1:] for i in candicate_nodes]

        return candicate_nodes,candicate_weights


    def Prob_Calculation(self, prev_act, Score_Type  , Score_Rate, Trace_Qty = 3 ,Basic_Score = 100   ):
        
        
        
        graph_list = []
        prev_qty = len(prev_act)
        weight = 1 
        candicate_weights = []
        result_dict = {}
        max_node = ''
        max_score = 0 
        
        
        for g in self.list_G:

            graph_list.append(list(g.nodes)[0])

        
        if prev_qty == 0:
            result_dict = {}
            max_score = 0
            max_node = ''
            
        else:
#             print(prev_act, Score_Type  , Score_Rate, Trace_Qty ,Basic_Score  )
#             ['11', '8', '13'] 3 100 greedy 20
#             print(type(prev_act), type())
            
            if prev_qty > Trace_Qty:    
                prev_act = prev_act[-Trace_Qty:]
                

            # Start collecting the probability and the possible events by checking the previous graph depending on the level diff from the target (from level(-1) to -traceqty)
            for i in range(len(prev_act)):
                prev_newAct = prev_act[-(i+1):]
                initial_S = prev_newAct[0]
                g_idx = graph_list.index(initial_S)

                if len(prev_newAct) == 1: 
                    e = prev_newAct[0]
                    weight = 1

                else:
                    real_s = ''
                    real_e = ''

                    # Start tracing the graph 
                    for idx in range(len(prev_newAct)-1):
                        if idx == 0:
                            real_s = prev_newAct[0]
                            s = real_s
                        else: 
                            real_s = prev_newAct[idx]

                        real_e = prev_newAct[idx+1]
                        e = f'L{idx+1}_{real_s}-{real_e}'
                        
                        if e in self.list_G[g_idx].out_edges(s): 
                            weight *= self.list_G[g_idx].edges[s,e]['weight'] 
                        else:
                            weight = 0
                            
                        s = e
                
                if weight == 0: 
                    continue
                else: 
                    weight = round(weight,2)

                ##### Getting the candidate nodes with probability #####
                tmp_result_nodes, tmp_result_prob = self.result_nodewithProb(g_idx,e)

                ##### Score Calculation #####
                # Score_Type = ['greedy', 'equally' ]
                if Score_Type == 'greedy':
                    if Score_Rate < 0:
                        Score_Rate = -Score_Rate
                        tmp_score = [round( Basic_Score * p * weight * - (Score_Rate**i), 5) for p in  tmp_result_prob] 
                    
                    else:
                        tmp_score = [round( Basic_Score * p * weight * (Score_Rate**i), 5) for p in  tmp_result_prob] 

                elif Score_Type == 'equally':
                    tmp_score = [round( Basic_Score * p * weight * (Score_Rate*(i+1)), 5) for p in  tmp_result_prob] 

                keys = list(result_dict.keys()) 
                
                for node, score in zip(tmp_result_nodes, tmp_score):
                    
                    if node in keys: 
                        result_dict[node] = result_dict[node]+score 
                        
                    else:
                        result_dict[node] = score
                
                
                max_score = max(result_dict.items(), key=operator.itemgetter(1))[1]
                max_node = max(result_dict.items(), key=operator.itemgetter(1))[0]


        return  max_node, max_score
    
    
    def unpack_list(self,x):
    
        [a] = x
        return a
    
    
    def unpack_tuples(self, result, idx):
        
        (node, score) = result
        if idx == 0:
            return str(node)
        else:
            return score 
    
    
    def make_RecapDataFrame(self ,Trace_Qty, Score_Type , Score_Rate,   Basic_Score = 100  ):
        
        list_activities, prev_act,last_act  = self.activity_seperation()
        
        print( Score_Type  , Score_Rate, Trace_Qty ,Basic_Score )
        result = prev_act.apply(lambda x: self.Prob_Calculation(x, Score_Type, Score_Rate, Trace_Qty   , Basic_Score))
        
        last_act_ = self.last_act.apply(lambda x: self.unpack_list(x))
        
        df_result = pd.DataFrame(last_act_)
        df_result.columns = ['actual_Next']
        df_result.loc[:,'pred_Next'] =  result.apply(lambda x: self.unpack_tuples(x,0))
        df_result.loc[:,'pred_Score'] =  result.apply(lambda x: self.unpack_tuples(x,1))
        
        ##### Remove rows without the predicted result #####      
#         df_result['pred_Next'].replace('', np.nan, inplace = True)
#         df_resultwithoutNull = df_result.dropna(subset=['pred_Next'], how = 'all', axis = 0)
#         df_resultwithoutNull.reset_index(inplace = True, drop = True)
        
        return df_result

### Helpdesk Test 1-1. trace_qty = 3 with the respective graph 
    * Trained Graphs produced by the test data
        - Use the same evnet log for producing graphs and predicting the next event. 
        - If helpdesk_3(1 case = 3 events + 1 event) produces graphs for each unique envents, 
        then, predict the last label though 3 events in helpdesk_3.  

In [30]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'helpdesk{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

for file in files:
    print(file)
    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 3
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 3:
            trace_qty = 3

    target = Graph(file)
    g_list, act_list= target.graphs()
#     target.Draw_Graphs()
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)

helpdesk_result = df_comparison
helpdesk_result

helpdesk_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_4
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_5
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_7
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_10
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
eq

,accuracy_greedy_s(20),accuracy_greedy_s(10),accuracy_greedy_s(1),accuracy_greedy_s(-10),accuracy_greedy_s(-20),accuracy_equally_s(20),accuracy_equally_s(10),accuracy_equally_s(1),accuracy_equally_s(-10),accuracy_equally_s(-20),cnt
file,,,,,,,,,,,
helpdesk_3,37.83,37.83,37.83,2.62,2.62,37.83,37.83,37.83,2.62,2.62,2477.0
helpdesk_4,36.79,36.79,36.79,6.54,6.54,36.79,36.79,36.79,6.54,6.54,1117.0
helpdesk_5,41.02,41.02,41.02,11.43,11.43,41.02,41.02,41.02,11.43,11.43,490.0
helpdesk_7,47.52,47.52,47.52,8.91,8.91,47.52,47.52,47.52,8.91,8.91,101.0
helpdesk_10,53.33,53.33,53.33,20.00,20.00,53.33,53.33,53.33,20.00,20.00,15.0
helpdesk,86.17,86.17,86.17,0.00,0.00,86.17,86.17,86.17,0.00,0.00,3803.0


In [21]:
pd.DataFrame(helpdesk_result).to_csv("helpdesk_result(trace_qty_3).csv", index= None)

### Helpdesk Test 1-2. trace_qty = 7 with the respective graph 
    * Trained Graphs produced by the test data
        - Use the same evnet log for producing graphs and predicting the next event. 
        - If helpdesk_3(1 case = 3 events + 1 event) produces graphs for each unique envents, 
        then, predict the last label though 3 events in helpdesk_3.  

In [22]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'helpdesk{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

for file in files:
    print(file)
    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 7
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 7:
            trace_qty = 7

    target = Graph(file)
    g_list, act_list= target.graphs()
#     target.Draw_Graphs()
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)

helpdesk_result2 = df_comparison
helpdesk_result2

helpdesk_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_4
greedy 20 4 100
greedy 10 4 100
greedy 1 4 100
greedy -10 4 100
greedy -20 4 100
equally 20 4 100
equally 10 4 100
equally 1 4 100
equally -10 4 100
equally -20 4 100
helpdesk_5
greedy 20 5 100
greedy 10 5 100
greedy 1 5 100
greedy -10 5 100
greedy -20 5 100
equally 20 5 100
equally 10 5 100
equally 1 5 100
equally -10 5 100
equally -20 5 100
helpdesk_7
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
helpdesk_10
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
helpdesk
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
eq

,accuracy_greedy_s(20),accuracy_greedy_s(10),accuracy_greedy_s(1),accuracy_greedy_s(-10),accuracy_greedy_s(-20),accuracy_equally_s(20),accuracy_equally_s(10),accuracy_equally_s(1),accuracy_equally_s(-10),accuracy_equally_s(-20),cnt
file,,,,,,,,,,,
helpdesk_3,37.83,37.83,37.83,37.83,37.83,37.83,37.83,37.83,2.62,2.62,2477.0
helpdesk_4,36.79,36.79,36.79,36.79,36.79,36.79,36.79,36.79,6.54,6.54,1117.0
helpdesk_5,41.02,41.02,41.02,41.02,41.02,41.02,41.02,41.02,11.43,11.43,490.0
helpdesk_7,47.52,47.52,47.52,47.52,47.52,47.52,47.52,47.52,8.91,8.91,101.0
helpdesk_10,53.33,53.33,53.33,53.33,53.33,53.33,53.33,53.33,20.00,20.00,15.0
helpdesk,86.17,86.17,86.17,86.17,86.17,86.17,86.17,86.17,0.00,0.00,3803.0


In [23]:
pd.DataFrame(helpdesk_result).to_csv("helpdesk_result(trace_qty_7).csv", index= None)

### Helpdesk Test 2-1. trace_qty = 3 with the graph produced by the original data.
    * Trained Graphs produced by the original data
        - Use the different evnet log dataset for producing graphs and predicting the next event.
        - Produce the graph by using the original data. 
        - Predict the nest events by referring the graph by the original data. 
        - If the original log 'helpdesk' produces graphs for each unique envents, 
            then, predict the last label though tracing 3 events in helpdesk_3.  

In [36]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'helpdesk{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

target = Graph('helpdesk')
g_list, _ = target.graphs()


for file in files:

    print(file)

    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 3
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 3:
            trace_qty =3
        
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    
    target2 = Graph(file)
    act_list = target2.create_listEdges()
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)
    
helpdesk_byOriginal = df_comparison

helpdesk_byOriginal

helpdesk_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_4
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_5
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_7
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_10
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
eq

,accuracy_greedy_s(20),accuracy_greedy_s(10),accuracy_greedy_s(1),accuracy_greedy_s(-10),accuracy_greedy_s(-20),accuracy_equally_s(20),accuracy_equally_s(10),accuracy_equally_s(1),accuracy_equally_s(-10),accuracy_equally_s(-20),cnt
file,,,,,,,,,,,
helpdesk_3,65.81,65.81,65.81,2.66,2.66,65.81,65.81,65.81,2.66,2.66,2477.0
helpdesk_4,73.14,73.14,73.14,1.43,1.43,73.14,73.14,73.14,1.43,1.43,1117.0
helpdesk_5,72.24,72.24,72.24,0.82,0.82,72.24,72.24,72.24,0.82,0.82,490.0
helpdesk_7,10.89,10.89,10.89,4.95,4.95,10.89,10.89,10.89,4.95,4.95,101.0
helpdesk_10,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.0
helpdesk,86.17,86.17,86.17,0.00,0.00,86.17,86.17,86.17,0.00,0.00,3803.0


In [37]:
pd.DataFrame(helpdesk_byOriginal).to_csv("helpdesk_byOriginal(trace_qty_3).csv", index= None)

### Helpdesk Test 2-2. trace_qty = 7 with the graph produced by the original data.
    * Trained Graphs produced by the original data
        - Use the different evnet log dataset for producing graphs and predicting the next event.
        - Produce the graph by using the original data. 
        - Predict the nest events by referring the graph by the original data. 
        - If the original log 'helpdesk' produces graphs for each unique envents, 
            then, predict the last label though tracing 3 events in helpdesk_3.  

In [39]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'helpdesk{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

target = Graph('helpdesk')
g_list, _ = target.graphs()


for file in files:

    print(file)

    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 7
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 7:
            trace_qty =7
        
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    
    target2 = Graph(file)
    act_list = target2.create_listEdges()
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)
    
helpdesk_byOriginal_7 = df_comparison

pd.DataFrame(helpdesk_byOriginal_7).to_csv("helpdesk_byOriginal(trace_qty_7).csv", index= None)

helpdesk_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
helpdesk_4
greedy 20 4 100
greedy 10 4 100
greedy 1 4 100
greedy -10 4 100
greedy -20 4 100
equally 20 4 100
equally 10 4 100
equally 1 4 100
equally -10 4 100
equally -20 4 100
helpdesk_5
greedy 20 5 100
greedy 10 5 100
greedy 1 5 100
greedy -10 5 100
greedy -20 5 100
equally 20 5 100
equally 10 5 100
equally 1 5 100
equally -10 5 100
equally -20 5 100
helpdesk_7
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
helpdesk_10
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
helpdesk
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
eq

### Hospitalbilling Test 1-1. trace_qty = 3 with the respective graph 
    * Trained Graphs produced by the test data
        - Use the same evnet log for producing graphs and predicting the next event. 
        - If helpdesk_3(1 case = 3 events + 1 event) produces graphs for each unique envents, 
        then, predict the last label though 3 events in helpdesk_3.  

In [42]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'hospitalbilling{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

for file in files:
    print(file)
    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 3
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 3:
            trace_qty = 3

    target = Graph(file)
    g_list, act_list= target.graphs()
#     target.Draw_Graphs()
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)

hospitalbilling_result_3 = df_comparison
pd.DataFrame(hospitalbilling_result_3).to_csv("hospitalbilling_result(trace_qty_3).csv")
print(hospitalbilling_result_3)

hospitalbilling_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_4
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_5
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_7
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_10
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 

### hospitalbilling Test 1-2. trace_qty = 7 with the respective graph 
    * Trained Graphs produced by the test data
        - Use the same evnet log for producing graphs and predicting the next event. 
        - If hospitalbilling_3(1 case = 3 events + 1 event) produces graphs for each unique envents, 
        then, predict the last label though 3 events in hospitalbilling_3.  

In [43]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'hospitalbilling{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

for file in files:
    print(file)
    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 7
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 7:
            trace_qty = 7

    target = Graph(file)
    g_list, act_list= target.graphs()
#     target.Draw_Graphs()
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)

hospitalbilling_result_7 = df_comparison

pd.DataFrame(hospitalbilling_result_7).to_csv("hospitalbilling_result(trace_qty_7).csv")
print(hospitalbilling_result_7)

hospitalbilling_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_4
greedy 20 4 100
greedy 10 4 100
greedy 1 4 100
greedy -10 4 100
greedy -20 4 100
equally 20 4 100
equally 10 4 100
equally 1 4 100
equally -10 4 100
equally -20 4 100
hospitalbilling_5
greedy 20 5 100
greedy 10 5 100
greedy 1 5 100
greedy -10 5 100
greedy -20 5 100
equally 20 5 100
equally 10 5 100
equally 1 5 100
equally -10 5 100
equally -20 5 100
hospitalbilling_7
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
hospitalbilling_10
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
hospitalbilling
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 

### Helpdesk Test 2-1. trace_qty = 3 with the graph produced by the original data.
    * Trained Graphs produced by the original data
        - Use the different evnet log dataset for producing graphs and predicting the next event.
        - Produce the graph by using the original data. 
        - Predict the nest events by referring the graph by the original data. 
        - If the original log 'helpdesk' produces graphs for each unique envents, 
            then, predict the last label though tracing 3 events in helpdesk_3.  

In [48]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'hospitalbilling{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

target = Graph('hospitalbilling')
g_list, _ = target.graphs()


for file in files:

    print(file)

    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 3
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 3:
            trace_qty =3
        
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    
    target2 = Graph(file)
    act_list = target2.create_listEdges()
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)
    
hospitalbilling_byOriginal = df_comparison

pd.DataFrame(hospitalbilling_byOriginal).to_csv("hospitalbilling_byOriginal(trace_qty_3).csv")
print(hospitalbilling_byOriginal)

hospitalbilling_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_4
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_5
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_7
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_10
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 

### Helpdesk Test 2-2. trace_qty = 7 with the graph produced by the original data.
    * Trained Graphs produced by the original data
        - Use the different evnet log dataset for producing graphs and predicting the next event.
        - Produce the graph by using the original data. 
        - Predict the nest events by referring the graph by the original data. 
        - If the original log 'helpdesk' produces graphs for each unique envents, 
            then, predict the last label though tracing 3 events in helpdesk_3.  

In [49]:
sub_name = ['_3', '_4', '_5', '_7', '_10', '']


files =  [f'hospitalbilling{j}'  for j in sub_name]


df_comparison = pd.DataFrame(files, columns = ['file'])
df_comparison.set_index('file', drop = True, inplace = True)

target = Graph('hospitalbilling')
g_list, _ = target.graphs()


for file in files:

    print(file)

    prefix = file.split('_')

    if len(prefix) == 1:
        trace_qty = 7
    else: 
        trace_qty = int(prefix[1])
        if trace_qty > 7:
            trace_qty =7
        
    Score_List = [20, 10, 1 , -10, -20 ]
    Score_Type = ['greedy', 'equally' ]
    
    target2 = Graph(file)
    act_list = target2.create_listEdges()
    pred = prediction(g_list, act_list)
    
    for s_type in Score_Type: 
        for s in Score_List: 
            df_result= pred.make_RecapDataFrame(Trace_Qty = trace_qty, Score_Type = s_type, Score_Rate = s)
            col_ac_name = f'accuracy_{s_type}_s({s})'
            df_comparison.loc[file, col_ac_name] = round(accuracy_score(df_result['actual_Next'] , df_result['pred_Next'], normalize = True), 4) * 100
    df_comparison.loc[file, f'cnt'] = len(df_result)
    
hospitalbilling_byOriginal_7 = df_comparison

pd.DataFrame(hospitalbilling_byOriginal_7).to_csv("hospitalbilling_byOriginal(trace_qty_7).csv")
print(hospitalbilling_byOriginal_7)

hospitalbilling_3
greedy 20 3 100
greedy 10 3 100
greedy 1 3 100
greedy -10 3 100
greedy -20 3 100
equally 20 3 100
equally 10 3 100
equally 1 3 100
equally -10 3 100
equally -20 3 100
hospitalbilling_4
greedy 20 4 100
greedy 10 4 100
greedy 1 4 100
greedy -10 4 100
greedy -20 4 100
equally 20 4 100
equally 10 4 100
equally 1 4 100
equally -10 4 100
equally -20 4 100
hospitalbilling_5
greedy 20 5 100
greedy 10 5 100
greedy 1 5 100
greedy -10 5 100
greedy -20 5 100
equally 20 5 100
equally 10 5 100
equally 1 5 100
equally -10 5 100
equally -20 5 100
hospitalbilling_7
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
hospitalbilling_10
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 7 100
greedy -20 7 100
equally 20 7 100
equally 10 7 100
equally 1 7 100
equally -10 7 100
equally -20 7 100
hospitalbilling
greedy 20 7 100
greedy 10 7 100
greedy 1 7 100
greedy -10 